In [11]:
from datetime import datetime, timedelta, UTC
from functools import partial

from sqlmodel import Session, select, col
import pandas as pd

from clue_access import BLEDevice, QTDevice, run_in_session

In [ ]:
from collections.abc import Iterable


def find_qt_devices_by_name(session: Session, name: str) -> Iterable[tuple[QTDevice, datetime]]:
    statement = (
        select(QTDevice, BLEDevice.time)
        .join(BLEDevice, col(QTDevice.ble_device_id) == col(BLEDevice.id))
        .where(QTDevice.name == name, BLEDevice.time > datetime.now(UTC) - timedelta(hours=24))
        .order_by(col(BLEDevice.time).desc())
    )
    data = session.exec(statement).all()
    return data


data = list(run_in_session(partial(find_qt_devices_by_name, name="QT 2013036")))
len(data)


45440

In [13]:
df = pd.DataFrame(device.model_dump() | {"time": time} for device, time in data)
df

,mac,mode,name,snowmode,ble_device_id,color,armed,vbat,time
0,34:81:F4:9E:B7:6C,0,QT 2013036,False,6758603,2,False,12.42,2025-01-14 02:46:01.641
1,34:81:F4:9E:B7:6C,0,QT 2013036,False,6758579,2,False,12.42,2025-01-14 02:46:01.582
2,34:81:F4:9E:B7:6C,0,QT 2013036,False,6758518,2,False,12.42,2025-01-14 02:46:01.490
3,34:81:F4:9E:B7:6C,0,QT 2013036,False,6758471,2,False,12.42,2025-01-14 02:46:01.435
4,34:81:F4:9E:B7:6C,0,QT 2013036,False,6758445,2,False,12.42,2025-01-14 02:46:01.416
...,...,...,...,...,...,...,...,...,...
45435,34:81:F4:9E:B7:6C,0,QT 2013036,False,298485,2,False,12.24,2025-01-13 05:52:25.569
45436,34:81:F4:9E:B7:6C,0,QT 2013036,False,298444,2,False,12.24,2025-01-13 05:52:25.531
45437,34:81:F4:9E:B7:6C,0,QT 2013036,False,298425,2,False,12.24,2025-01-13 05:52:25.473
45438,34:81:F4:9E:B7:6C,0,QT 2013036,False,298392,2,False,12.24,2025-01-13 05:52:25.431


In [14]:
import plotly.express as px

fig = px.scatter(df, x="time", y="vbat", title="Battery Voltage Over Time")
fig.update_layout(xaxis_title="Time", yaxis_title="Battery Voltage (vbat)", xaxis=dict(tickangle=90))
fig.show()
